# Inverse Kinematics as Optimization (to be merged)

TODO: merge with previous. The previous doc explained features and how they define objectives of a constrained optimization problem. Here we show how to use this to solve IK optimization problems.

At the bottom there is more general text explaining the basic concepts.

## Demo of features in Inverse Kinematics

Let's setup a standard configuration. (Lock the window with "Always on Top".)

In [ ]:
import sys
sys.path.append('../build') #rai/lib')
import numpy as np
from robotic import ry

In [ ]:
C = ry.Config()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../rai-robotModels/objects/kitchen.g')
C.view()

For simplicity, let's add a frame that represents goals

In [ ]:
goal = C.addFrame("goal")
goal.setShape(ry.ST.sphere, [.05])
goal.setColor([.5,1,1])
goal.setPosition([1,.5,1])
X0 = C.getFrameState() #store the initial configuration

We create an IK engine. The only objective is that the `positionDiff` (position difference in world coordinates) between `pr2L` (the yellow blob in the left hand) and `goal` is equal to zero:

In [ ]:
IK = C.komo_IK(False)
IK.addObjective([], type=ry.OT.eq, feature=ry.FS.positionDiff, frames=['pr2L', 'goal'])

We now call the optimizer (True means with random initialization/restart).

In [ ]:
IK.optimize()
IK.getReport()

The best way to retrieve the result is to copy the optimized IK configuration back into your working configuration C, which is now also displayed

In [ ]:
C.setFrameState( IK.getFrameState(0) )

We can redo the optimization, but for a different configuration, namely a configuration where the goal is in another location.
For this we move goal in our working configuration C, then copy C back into the IK engine's configurations:

In [ ]:
## (iterate executing this cell for different goal locations!)

# move goal
goal.setPosition([.8,.2,1.5])

# # copy C into the IK's internal configuration(s)
# IK.setConfigurations(C)

# redefine the IK problem
IK = C.komo_IK(False)
IK.addObjective([], type=ry.OT.eq, feature=ry.FS.positionDiff, frames=['pr2L', 'goal'])

# reoptimize
IK.optimize(0.) # 0: no adding of noise for a random restart
print(IK.getReport())

# grab result
C.setFrameState( IK.getFrameState(0) )

Let's solve some other problems, always creating a novel IK engine:

The relative position of `goal` in `pr2R` coordinates equals [0,0,-.2] (which is 20cm straight in front of the yellow blob)

In [ ]:
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective([],type=ry.OT.eq, feature=ry.FS.positionRel, frames=['goal','pr2R'], target=[0,0,-.2])
IK.optimize()
C.setFrameState( IK.getFrameState(0) )

The distance between `pr2R` and `pr2L` is zero:

In [ ]:
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective([], type=ry.OT.eq, feature=ry.FS.distance, frames=['pr2L','pr2R'])
IK.optimize()
C.setFrameState( IK.getFrameState(0) )

The 3D difference between the z-vector of `pr2R` and the z-vector of `goal`:

In [ ]:
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective([], type=ry.OT.eq, feature=ry.FS.vectorZDiff, frames=['pr2R', 'goal'])
IK.optimize()
C.setFrameState( IK.getFrameState(0) )

The scalar product between the z-vector of `pr2R` and the z-vector of `goal` is zero:

In [ ]:
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective([], type=ry.OT.eq, feature=ry.FS.scalarProductZZ, frames=['pr2R', 'goal'])
IK.optimize()
C.setFrameState( IK.getFrameState(0) )

etc etc

In [ ]:
# Designing a cylinder grasp

In [ ]:
D=0
C=0

In [ ]:
C = ry.Config()
D = C.view()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../rai-robotModels/objects/kitchen.g')
C.setJointState([.7], ["l_gripper_l_finger_joint"])

In [ ]:
goal = C.addFrame("goal")
goal.setShape(ry.ST.cylinder, [0,0,.2, .03])
goal.setColor([.5,1,1])
goal.setPosition([1,.5,1])
X0 = C.getFrameState()

In [ ]:
C.setFrameState(X0)
goal.setPosition([1,.5,1.2])
IK = C.komo_IK(False)
IK.addObjective([], type=ry.OT.eq,   feature=ry.FS.positionDiff, frames=['pr2L', 'goal'], scale=[[1,0,0],[0,1,0]])
IK.addObjective([], type=ry.OT.ineq, feature=ry.FS.positionDiff, frames=['pr2L', 'goal'], scale=[[0,0,1]], target=[0,0,.05])
IK.addObjective([], type=ry.OT.ineq, feature=ry.FS.positionDiff, frames=['pr2L', 'goal'], scale=[[0,0,-1]], target=[0,0,-.05])
IK.addObjective([], type=ry.OT.sos,  feature=ry.FS.scalarProductZZ, frames=['pr2L', 'goal'], scale=[0.1])
IK.addObjective([], type=ry.OT.eq,   feature=ry.FS.scalarProductXZ, frames=['pr2L', 'goal'])
IK.optimize()
C.setFrameState( IK.getFrameState(0) )
IK.getReport()